In [13]:
import random, copy
from pprint import pprint

class JadwalKuliah:
    def __init__(self, kode_matkul, kode_dosen, sks_akademik, kode_ruangan, hari, jam_mulai, jam_selesai, tipe_kelas):
        self.kode_matkul = kode_matkul
        self.kode_dosen = kode_dosen
        self.sks_akademik = sks_akademik
        self.kode_ruangan = kode_ruangan
        self.hari = hari
        self.jam_mulai = jam_mulai
        self.jam_selesai = jam_selesai
        self.tipe_kelas = tipe_kelas  # 'TEORI' atau 'PRAKTIKUM'

    def __repr__(self):
        return f"<{self.kode_matkul}, {self.kode_dosen}, {self.sks_akademik}, {self.kode_ruangan}, {self.hari}, {self.jam_mulai}, {self.jam_selesai}, {self.tipe_kelas}>\n"

# print([i for i in range(9, 13)])
# [9, 10, 11, 12]

BOBOT_PENALTI = {
    "dosen_bentrok": 15, # dosen mengajar 2/lebih kelas bersamaan
    "ruangan_bentrok": 15, # kelas digunakan 2/lebih mata kuliah bersamaan
    "asdos_nabrak_dosen": 15, # kelas dosen dan asistensi berjalan dalam waktu yang sama
    "dosen_terbebani": 10, # beban sks melebihi batas
    "weekend_class": 10, # kelas diadakan di hari Sabtu
    "istirahat": 5, # kelas diadakan di jam istirahat
    "salah_tipe_ruangan": 3, # tipe ruangan salah (PRAKTIKUM di TEORI)
    "diluar_jam_kerja": 10, # as it says
    "melanggar_preferensi": 5 # as it says
}

# List mata kuliah ( + jumlah mahasiswa )
matakuliah_list = [
  # tahun 1
  { "kode": "TC511", "nama": "MATEMATIKA DISKRIT", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], "jumlah_mahasiswa": 300},
  { "kode": "TC511-AS", "nama": "MATEMATIKA DISKRIT", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], "jumlah_mahasiswa": 300},
  { "kode": "TC512", "nama": "PENGANTAR TEKNOLOGI INFORMASI", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], "jumlah_mahasiswa": 300},
  { "kode": "TC512-AS", "nama": "PENGANTAR TEKNOLOGI INFORMASI", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "PRAKTIKUM", "bidang": [], "jumlah_mahasiswa": 300},
  { "kode": "TC513", "nama": "DASAR-DASAR PEMROGRAMAN", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "PRAKTIKUM", "bidang": [], "jumlah_mahasiswa": 300},
  { "kode": "TC513-AS", "nama": "DASAR-DASAR PEMROGRAMAN", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "PRAKTIKUM", "bidang": [], "jumlah_mahasiswa": 300},
  { "kode": "TC514", "nama": "BAHASA INGGRIS", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 3, "sks_bayar": 3, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], "jumlah_mahasiswa": 300},
  { "kode": "TC515", "nama": "SISTEM BASIS DATA", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], "jumlah_mahasiswa": 300},
  { "kode": "TC515-AS", "nama": "SISTEM BASIS DATA", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "PRAKTIKUM", "bidang": [], "jumlah_mahasiswa": 300},

  # tahun 2
  { "kode": "TC611", "nama": "KECERDASAN BUATAN", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], "jumlah_mahasiswa": 300 },
  { "kode": "TC611-AS", "nama": "KECERDASAN BUATAN", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "PRAKTIKUM", "bidang": [], "jumlah_mahasiswa": 300 },
  { "kode": "TC612", "nama": "PEMROGRAMAN WEB", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "PRAKTIKUM", "integrated_class": True, "bidang": [], "jumlah_mahasiswa": 300 },
  { "kode": "TC613", "nama": "PEMROGRAMAN BERORIENTASI OBJEK", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "PRAKTIKUM", "integrated_class": True, "bidang": [], "jumlah_mahasiswa": 300 },
  { "kode": "TC614", "nama": "DASAR PEMROGRAMAN JARINGAN", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], "jumlah_mahasiswa": 300 },
  { "kode": "TC615", "nama": "ORGANISASI DAN ARSITEKTUR KOMPUTER", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 3, "sks_bayar": 3, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], "jumlah_mahasiswa": 300 },

  # tahun 3 - SE
  { "kode": "TC712", "nama": "PEMROGRAMAN MOBILE", "kelompok": "BIDANG MINAT SOFTWARE ENGINEERING", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "PRAKTIKUM", "bidang": [], 'jumlah_mahasiswa': 130 },
  { "kode": "TC712-AS", "nama": "PEMROGRAMAN MOBILE", "kelompok": "BIDANG MINAT SOFTWARE ENGINEERING", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "PRAKTIKUM", "bidang": [], 'jumlah_mahasiswa': 130 },
  { "kode": "TC715", "nama": "MANAJEMEN PROYEK", "kelompok": "BIDANG MINAT SOFTWARE ENGINEERING", "sks_akademik": 3, "sks_bayar": 3, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], 'jumlah_mahasiswa': 130 },
  { "kode": "TC717", "nama": "PENGUJIAN SISTEM", "kelompok": "BIDANG MINAT SOFTWARE ENGINEERING", "sks_akademik": 3, "sks_bayar": 3, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], 'jumlah_mahasiswa': 130 },
  { "kode": "TC719", "nama": "PEMROGRAMAN BERORIENTASI PLATFORM", "kelompok": "BIDANG MINAT SOFTWARE ENGINEERING", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "PRAKTIKUM", "bidang": [], 'jumlah_mahasiswa': 130 },
  { "kode": "TC781", "nama": "SERTIFIKASI SOFTWARE ENGINEERING - ORACLE", "kelompok": "BIDANG MINAT SOFTWARE ENGINEERING", "sks_akademik": 3, "sks_bayar": 3, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], 'jumlah_mahasiswa': 130 },

  # tahun 3 - network
  { "kode": "TC751", "nama": "PERANCANGAN DAN MANAJEMEN JARINGAN", "kelompok": "BIDANG MINAT NETWORK ENGINEERING", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], 'jumlah_mahasiswa': 50 },
  { "kode": "TC754", "nama": "ROUTING DAN SWITCHING 1", "kelompok": "BIDANG MINAT NETWORK ENGINEERING", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "PRAKTIKUM", "bidang": [], 'jumlah_mahasiswa': 50 },
  { "kode": "TC757", "nama": "KEAMANAN JARINGAN", "kelompok": "BIDANG MINAT NETWORK ENGINEERING", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], 'jumlah_mahasiswa': 50 },
  { "kode": "TC783", "nama": "SERTIFIKASI NETWORK ENGINEERING - RSNA", "kelompok": "BIDANG MINAT NETWORK ENGINEERING", "sks_akademik": 3, "sks_bayar": 3, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], 'jumlah_mahasiswa': 50 },

  # tahun 3 - DS
  { "kode": "TC783", "nama": "SERTIFIKASI NETWORK ENGINEERING - RSNA", "kelompok": "BIDANG MINAT NETWORK ENGINEERING", "sks_akademik": 3, "sks_bayar": 3, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], 'jumlah_mahasiswa': 120 },
  { "kode": "TC733", "nama": "DEEP LEARNING", "kelompok": "BIDANG MINAT DATA SCIENCE", "sks_akademik": 3, "sks_bayar": 3, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], 'jumlah_mahasiswa': 120 },
  { "kode": "TC734", "nama": "PEMODELAN SPASIAL", "kelompok": "BIDANG MINAT DATA SCIENCE", "sks_akademik": 3, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], 'jumlah_mahasiswa': 120 },
  { "kode": "TC737", "nama": "PEMROSESAN BAHASA ALAMI", "kelompok": "BIDANG MINAT DATA SCIENCE", "sks_akademik": 3, "sks_bayar": 3, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], 'jumlah_mahasiswa': 120 },
  { "kode": "TC782", "nama": "SERTIFIKASI DATA SCIENCE - ORACLE", "kelompok": "BIDANG MINAT DATA SCIENCE", "sks_akademik": 3, "sks_bayar": 3, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], 'jumlah_mahasiswa': 120 },

  # # tahun 4
  # { "kode": "TC811","nama": "TUGAS TALENTA UNGGUL 1", "kelompok": "TALENTA UNGGUL", "sks_akademik": 4, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], 'jumlah_mahasiswa': 300 },
  # { "kode": "TC812","nama": "TUGAS TALENTA UNGGUL 2", "kelompok": "TALENTA UNGGUL", "sks_akademik": 4, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], 'jumlah_mahasiswa': 300 },
  # { "kode": "TC813","nama": "PUBLIKASI KARYA ILMIAH", "kelompok": "WAJIB PROGRAM STUDI", "sks_akademik": 4, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], 'jumlah_mahasiswa': 300 },
  # { "kode": "TC821", "nama": "KERJA PRAKTEK", "kelompok": "TALENTA UNGGUL", "sks_akademik": 8, "sks_bayar": 8, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], 'jumlah_mahasiswa': 300 },
  # { "kode": "TC822", "nama": "LAPORAN KERJA PRAKTEK", "kelompok": "TALENTA UNGGUL", "sks_akademik": 4, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], 'jumlah_mahasiswa': 300 },

  # { "kode": "TC833", "nama": "PEMBIMBINGAN INDUSTRI 3", "kelompok": "MERDEKA BELAJAR KAMPUS MERDEKA (MBKM)", "sks_akademik": 4, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], 'jumlah_mahasiswa': 300 },
  # { "kode": "TC841", "nama": "MBKM 1", "kelompok": "MERDEKA BELAJAR KAMPUS MERDEKA (MBKM)", "sks_akademik": 4, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], 'jumlah_mahasiswa': 300 },
  # { "kode": "TC842", "nama": "MBKM 2", "kelompok": "MERDEKA BELAJAR KAMPUS MERDEKA (MBKM)", "sks_akademik": 4, "sks_bayar": 4, "prodi": "S1 TEKNIK INFORMATIKA", "tipe_kelas": "TEORI", "bidang": [], 'jumlah_mahasiswa': 300 },
]

# List dosen
dosen_list = [
  { "nip": "67000", "nama": "KEN", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ["MATHEMATICS", "SOFTWARE ENGINEERING"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67001", "nama": "ALZ", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ["MATHEMATICS"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67002", "nama": "CHRISTINE", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ["INTELLIGENT SYSTEM"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67003", "nama": "DIAN", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ["NETWORKING AND COMMUNICATION"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67004", "nama": "EKO", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ["INFORMATION TECHNOLOGY"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67005", "nama": "EVANGS", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ["INTELLIGENT SYSTEM"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67006", "nama": "HENDRY", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ["INTELLIGENT SYSTEM"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67007", "nama": "HINDRIYANTO", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ["INTELLIGENT SYSTEM"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67008", "nama": "INDRASTANTI", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ["NETWORKING AND COMMUNICATION"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67009", "nama": "ADI", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67010", "nama": "INEKE", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67011", "nama": "NINA", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ["SOFTWARE ENGINEERING"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "68001", "nama": "OCSA", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67012", "nama": "RADIUS", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ["SOFTWARE ENGINEERING"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67013", "nama": "YULIANTO", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ["INFORMATION SYSTEM"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67014", "nama": "SUPRIHADI", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ["DIGITAL BUSINESS"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67015", "nama": "TEGUH", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ["NETWORKING AND COMMUNICATION"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67016", "nama": "THEOPHILUS", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ["NETWORKING AND COMMUNICATION"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67017", "nama": "WIWIN", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ["NETWORKING AND COMMUNICATION"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67018", "nama": "YEREMIA", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ["INFORMATION SYSTEM"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67019", "nama": "YOS", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ["SOFTWARE ENGINEERING"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "56001", "nama": "BUDHI", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": ["INFORMATION SYSTEM"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67020", "nama": "SUHARYADI", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "55001", "nama": "ARIYA", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67021", "nama": "RICHARD", "prodi": "S1 TEKNIK INFORMATIKA", "status": "DOSEN TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67022", "nama": "DANIEL", "prodi": "", "status": "DOSEN TETAP", "kepakaran": ["INFORMATION TECHNOLOGY"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67023", "nama": "IRWAN", "prodi": "", "status": "DOSEN TETAP", "kepakaran": ["NETWORKING AND COMMUNICATION"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67024", "nama": "YESSICA", "prodi": "", "status": "DOSEN TETAP", "kepakaran": ["MATHEMATICS"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67025", "nama": "ADE", "prodi": "", "status": "DOSEN TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67026", "nama": "KRISTOKO", "prodi": "", "status": "DOSEN TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67027", "nama": "FELIX", "prodi": "", "status": "DOSEN TETAP", "kepakaran": ["SOFTWARE ENGINEERING"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67028", "nama": "ARIE", "prodi": "", "status": "DOSEN TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67029", "nama": "ERWIEN", "prodi": "", "status": "DOSEN TETAP", "kepakaran": ["NETWORKING AND COMMUNICATION"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "55002", "nama": "WINARSO", "prodi": "", "status": "DOSEN TETAP", "kepakaran": ["INFORMATION TECHNOLOGY"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67030", "nama": "TEGUH", "prodi": "", "status": "DOSEN TETAP", "kepakaran": ["DIGITAL BUSINESS"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67031", "nama": "RISSAL", "prodi": "", "status": "DOSEN TETAP", "kepakaran": ["NETWORKING AND COMMUNICATION"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67032", "nama": "KRISMIYATI", "prodi": "", "status": "DOSEN TETAP", "kepakaran": ["EDUCATION"], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67033", "nama": "HANITA", "prodi": "", "status": "DOSEN TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67034", "nama": "CHARITAS", "prodi": "", "status": "DOSEN TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67035", "nama": "FAJAR", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67036", "nama": "RIKO", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67037", "nama": "CHERYL", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67038", "nama": "DANANG", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67039", "nama": "EDNA", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67040", "nama": "HILDA", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67041", "nama": "MARTZA", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "", "nama": "RAYNALDO", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "68002", "nama": "VIKKY", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67042", "nama": "VYORBIGGER", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67043", "nama": "ERYKA", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "68003", "nama": "NATALIA", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "", "nama": "WILSON", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "", "nama": "YULIA", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "", "nama": "CHRISTA", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "", "nama": "SEPTOVAN", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "67044", "nama": "ANGLING", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "11001", "nama": "DANIEL", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "", "nama": "AFIYATAR", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } },
  { "nip": "", "nama": "VIRGELIUS", "prodi": "", "status": "DOSEN TIDAK TETAP", "kepakaran": [], "preferensi": { "hindari_hari": [], "hindari_jam": [] } }
]

# List ruang (TEORI = T, PRAKTIKUM = P)
# {#     "kode": "", #     "tipe_ruangan": "", #     "kapasitas": 0, #     "plot": []# }
ruang_list = [
  { "kode": "FTI300", "tipe_ruangan": "TEORI", "kapasitas": 40, "plot": ["GENERAL"] },
  { "kode": "FTI301", "tipe_ruangan": "TEORI", "kapasitas": 19, "plot": ["S1 PERPUSTAKAAN DAN SAINS INFORMASI"] },
  { "kode": "FTI302", "tipe_ruangan": "TEORI", "kapasitas": 21, "plot": ["S1 HUBUNGAN MASYARAKAT"] },
  { "kode": "FTI306", "tipe_ruangan": "TEORI", "kapasitas": 43, "plot": ["GENERAL"] },
  { "kode": "FTI308", "tipe_ruangan": "TEORI", "kapasitas": 40, "plot": ["GENERAL"] },
  { "kode": "FTI329", "tipe_ruangan": "TEORI", "kapasitas": 44, "plot": ["GENERAL"] },
  { "kode": "FTI331", "tipe_ruangan": "TEORI", "kapasitas": 40, "plot": ["GENERAL"] },
  { "kode": "FTI333", "tipe_ruangan": "TEORI", "kapasitas": 56, "plot": ["GENERAL"] },
  { "kode": "FTI401", "tipe_ruangan": "TEORI", "kapasitas": 25, "plot": ["S2 MAGISTER SISTEM INFORMASI"] },
  { "kode": "FTI420", "tipe_ruangan": "TEORI", "kapasitas": 39, "plot": ["GENERAL"] },
  { "kode": "FTI422", "tipe_ruangan": "TEORI", "kapasitas": 45, "plot": ["GENERAL"] },
  { "kode": "FTI424", "tipe_ruangan": "TEORI", "kapasitas": 44, "plot": ["GENERAL"] },
  { "kode": "FTI504", "tipe_ruangan": "TEORI", "kapasitas": 40, "plot": ["S1 PENDIDIKAN TEKNIK INFORMATIKA DAN KOMPUTER"] },
  { "kode": "FTI505", "tipe_ruangan": "TEORI", "kapasitas": 21, "plot": ["GENERAL"] },
  { "kode": "FTI508", "tipe_ruangan": "TEORI", "kapasitas": 33, "plot": ["GENERAL"] },
  { "kode": "FTI511", "tipe_ruangan": "TEORI", "kapasitas": 40, "plot": ["GENERAL"] },

  { "kode": "FTI310", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 32, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI402", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 18, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI429", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 3, "plot": ["S1 DESAIN KOMUNIKASI VISUAL"], "specs": [] },
  { "kode": "FTI431", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 20, "plot": ["NETWORK", "CISCO"], "specs": [] },
  { "kode": "FTI451", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 27, "plot": ["SINARMAS"], "specs": [] },
  { "kode": "FTI453", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 7, "plot": ["D3 TEKNIK INFORMATIKA", "HARDWARE"], "specs": [] },
  { "kode": "FTI454", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 20, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI455", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 41, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI457", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 40, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI459", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 40, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI461", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 40, "plot": ["D3 TEKNIK INFORMATIKA"], "specs": [] },
  { "kode": "FTI463", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 40, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI464", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 40, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI465", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 40, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI467", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 40, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI469", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 40, "plot": ["GENERAL"], "specs": [] },
  { "kode": "FTI506", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 30, "plot": ["ORACLE"], "specs": [] },
  { "kode": "FTI507", "tipe_ruangan": "PRAKTIKUM", "kapasitas": 20, "plot": ["NETWORK", "CISCO"], "specs": [] },
  { "kode": "FTI400", "tipe_ruangan": "RAPAT", "kapasitas": 103, "plot": ["GENERAL"] }
]

# # testing
# pilihan_hari = ["Senin", "Selasa", "Rabu", "Kamis", "Jumat", "Sabtu"]
# pilihan_waktu = [ i for i in range(7, 19)] # 7-18
# dosen_s1_ti = [ d for d in dosen_list if d["prodi"] == "S1 TEKNIK INFORMATIKA"]
# bobot_dosen_pakar = [((len(set(d['kepakaran']) & set([]))*5) or 1) for d in dosen_s1_ti]
# dosen_pakar_s1_ti = [ d for d in dosen_s1_ti if len(set(d['kepakaran']) & set(['mathematics'.upper()])) > 0]
# ruangan_general_s1_ti = [r for r in ruang_list if "S1 TEKNIK INFORMATIKA" in r["plot"] or "GENERAL" in r["plot"]]
# ruangan_general_s1_ti_teori = [r for r in ruang_list if ("S1 TEKNIK INFORMATIKA" in r["plot"] or "GENERAL" in r["plot"]) and r['tipe_ruangan'] == "TEORI"]

# print(f"{'Jumlah Dosen S1 TI':<50}", len(dosen_s1_ti))
# print(f"{'Jumlah Dosen pakar matematika S1 TI':<50}", len(dosen_pakar_s1_ti))
# print(random.choices(population=dosen_s1_ti, weights=bobot_dosen_pakar, k=1)[0])
# for x in range(2):
#     print(random.choices(population=dosen_s1_ti, weights=bobot_dosen_pakar, k=1)[0])

# print(f"{'Jumlah Ruangan FTI prodi S1 TI + General':<50}", len(ruangan_general_s1_ti))
# print([len(set(r['plot']) & set(['MATHEMATICS', 'SOFTWARE ENGINEERING'])) + (int(r['kapasitas']) if r['tipe_ruangan'] != 'RAPAT' else 0) for r in ruang_list if ('S1 TEKNIK INFORMATIKA' or 'GENERAL' in r['plot'])])
# print(f"{'Jumlah Ruangan FTI prodi S1 TI + General #teori':<50}", len(ruangan_general_s1_ti_teori))

# print(f"{'Jumlah Dosen S1 TI':<50}", [hari for hari in pilihan_hari if hari not in dosen_list[0]['preferensi']['hindari_hari']])
# print(f"{'Kapasitas Ruangan index 0':<50}", ruang_list[0]['kapasitas'])

In [ ]:
# # Belum Dicek Belum Dipakai
# def repair_jadwal(jadwal, dosen_list, ruang_list, hari_list, jam_list):
#     # 1. Perbaiki bentrok dosen
#     for i, sesi1 in enumerate(jadwal):
#         for j, sesi2 in enumerate(jadwal):
#             if i != j and sesi1.hari == sesi2.hari and sesi1.jam == sesi2.jam:
#                 if sesi1.id_dosen == sesi2.id_dosen:
#                     sesi2.jam = random.choice([j for j in jam_list if j != sesi1.jam])

#     # 2. Pindahkan sesi dari hari Sabtu jika dosen mengajar
#     for sesi in jadwal:
#         if sesi.hari == "Sabtu" and sesi.tipe_kelas == "TEORI":
#             sesi.hari = random.choice([h for h in hari_list if h != "Sabtu"])

#     # 3. Pindahkan PRAKTIKUM dari ruang TEORI jika memungkinkan
#     for sesi in jadwal:
#         if sesi.tipe_kelas == "PRAKTIKUM" and sesi.ruang.startswith("T"):
#             ruang_praktikum = [r['id'] for r in ruang_list if r['id'].startswith("P")]
#             if ruang_praktikum:
#                 sesi.ruang = random.choice(ruang_praktikum)

#     # 4. Perbaiki beban SKS dosen
#     sks_dosen = {}
#     for sesi in jadwal:
#         sks_dosen[sesi.id_dosen] = sks_dosen.get(sesi.id_dosen, 0) + 1

#     for sesi in jadwal:
#         if sks_dosen[sesi.id_dosen] > 12:
#             dosen_tersedia = [d['id'] for d in dosen_list if d['id'] != sesi.id_dosen and sks_dosen.get(d['id'], 0) < 12]
#             if dosen_tersedia:
#                 sesi.id_dosen = random.choice(dosen_tersedia)

#     return jadwal

def repair_jadwal(jadwal, dosen_list, ruang_list):
    pilihan_hari = ["Senin", "Selasa", "Rabu", "Kamis", "Jumat"]
    pilihan_jam = list(range(7, 19))
    ruang_praktikum = [r for r in ruang_list if r['tipe_ruangan'] == 'PRAKTIKUM']
    ruang_teori = [r for r in ruang_list if r['tipe_ruangan'] == 'TEORI']

    # Atasi bentrok waktu dosen
    dosen_jadwal = {}
    for sesi in jadwal:
        if sesi.kode_dosen not in dosen_jadwal:
            dosen_jadwal[sesi.kode_dosen] = []
        
        conflict = False
        for sesi_lain in dosen_jadwal[sesi.kode_dosen]:
            if sesi.hari == sesi_lain['hari']:
                if not (sesi.jam_mulai >= sesi_lain['jam_selesai'] or sesi.jam_selesai <= sesi_lain['jam_mulai']):
                    conflict = True
                    break
        
        if conflict:
            sesi.hari = random.choice(pilihan_hari)
            sesi.jam_mulai = random.choice(pilihan_jam)
            sesi.jam_selesai = sesi.jam_mulai + sesi.sks_akademik
        
        dosen_jadwal[sesi.kode_dosen].append({'hari': sesi.hari, 'jam_mulai': sesi.jam_mulai, 'jam_selesai': sesi.jam_selesai})

    # Perbaiki praktikum di ruang teori
    for sesi in jadwal:
        if sesi.tipe_kelas == "PRAKTIKUM":
            if any(r['kode'] == sesi.kode_ruangan and r['tipe_ruangan'] == 'TEORI' for r in ruang_list):
                if ruang_praktikum:
                    sesi.kode_ruangan = random.choice(ruang_praktikum)['kode']

    # Pastikan tidak ada kelas teori di lab praktikum
    for sesi in jadwal:
        if sesi.tipe_kelas == "TEORI":
            if any(r['kode'] == sesi.kode_ruangan and r['tipe_ruangan'] == 'PRAKTIKUM' for r in ruang_list):
                if ruang_teori:
                    sesi.kode_ruangan = random.choice(ruang_teori)['kode']
    
    return jadwal


In [ ]:
def angka_ke_huruf(n):
    if 1 <= n <= 26:
        return chr(64 + n)  # Karena ord('A') = 65
    else:
        return None  # Diluar jangkauan 1-26

def generate_jadwal(matakuliah_list, dosen_list, ruang_list):
    jadwal = []
    pilihan_hari_dosen = ["SENIN", "SELASA", "RABU", "KAMIS", "JUMAT"]
    pilihan_hari_asisten = copy.deepcopy(pilihan_hari_dosen)
    pilihan_hari_asisten.extend(["SABTU"])

    jadwal_dosen = {}    # {nip: [ {hari, jam_mulai, jam_selesai}, ... ]}
    jadwal_ruangan = {}  # {kode_ruangan: [ {hari, jam_mulai, jam_selesai}, ... ]}
    beban_dosen = {} # {nip: beban_sks}
    beban_dosen_sudah_maksimal = []

    max_attempt = 10

    for matkul in matakuliah_list:
        # DOSEN (reminder: dosen tidak tetap belum tersentuh)
        dosen_prodi = [d for d in dosen_list if d["prodi"] == matkul['prodi']]
        bobot_dosen_berdasarkan_pakar = [(len(set(d['kepakaran']) & set(matkul['bidang']))*100 or 1) for d in dosen_prodi]
        
        # RUANGAN
        ruangan_prodi = [r for r in ruang_list if (matkul['prodi'] or 'GENERAL' in r['plot'])]
        bobot_ruangan_berdasarkan_plot_kapasitas_tipe = [(len(set(r['plot']) & set(matkul['bidang']))*5 or 1) + (int(r['kapasitas']) if r['tipe_ruangan'] != 'RAPAT' else 0) * (5 if r['tipe_ruangan'] == matkul['tipe_kelas'] else 1) for r in ruangan_prodi]
        
        jumlah_mahasiswa = matkul['jumlah_mahasiswa']
        index_kelas = 1
        
        while jumlah_mahasiswa > 0:
            ruang = random.choices(
                population=ruangan_prodi, 
                weights=bobot_ruangan_berdasarkan_plot_kapasitas_tipe, 
                k=1)[0]
            
            sukses = False
            attempt = 0
            while not sukses and max_attempt >= attempt:
                dosen = random.choices(
                    population=dosen_prodi, 
                    weights=bobot_dosen_berdasarkan_pakar, 
                    k=1)[0]

                if dosen['nip'] in beban_dosen and beban_dosen[dosen['nip']] <= (12 - matkul['sks_akademik']): 
                    # cek agar beban sks tidak melebihi 12
                    # print(f"beban sks {dosen['nip']} {beban_dosen[dosen['nip']]}")
                    sukses = True
                elif dosen['nip'] not in beban_dosen: 
                    beban_dosen[dosen['nip']] = 0

                attempt += 1

            if dosen['nip'] not in jadwal_dosen: jadwal_dosen[dosen['nip']] = []
            preferensi_hari = [hari for hari in pilihan_hari_dosen if hari not in dosen['preferensi']['hindari_hari']]
            hari = random.choice(preferensi_hari) if matkul['kode'][-3:] != '-AS' else random.choice(pilihan_hari_asisten)

            sukses = False
            attempt = 0
            while not sukses and max_attempt >= attempt:
                if ruang['kode'] not in jadwal_ruangan: jadwal_ruangan[ruang['kode']] = []

                jadwal_dosen_sesuai_hari = [d for d in jadwal_dosen[dosen['nip']] if d['hari'] == hari]
                jadwal_ruangan_sesuai_hari = [d for d in jadwal_ruangan[ruang['kode']] if d['hari'] == hari]

                if jadwal_ruangan_sesuai_hari:
                    list_jam_selesai_ruangan = [r['jam_selesai'] for r in jadwal_ruangan_sesuai_hari]
                    jam_terakhir_ruangan = max(list_jam_selesai_ruangan)
                    jam_mulai = jam_terakhir_ruangan if jam_terakhir_ruangan <= (18 - matkul['sks_akademik']) else 0
                else:
                    jam_mulai = 7

                if jam_mulai != 0:
                    jam_selesai = jam_mulai + matkul['sks_akademik']
                    sukses = True
                else:
                    hari = random.choice(preferensi_hari)

                if jadwal_dosen_sesuai_hari:
                    for jd in jadwal_dosen_sesuai_hari:
                        if jd['jam_mulai'] < jam_mulai < jd['jam_selesai'] or jd['jam_mulai'] < jam_mulai < jd['jam_selesai']:
                            ruang = random.choices(
                                population=ruangan_prodi, 
                                weights=bobot_ruangan_berdasarkan_plot_kapasitas_tipe, 
                                k=1)[0]
                            sukses = False

                attempt += 1

            sesi = JadwalKuliah(
                kode_matkul=f"{matkul['kode']}-{angka_ke_huruf(index_kelas)}",
                kode_dosen=dosen['nip'],
                sks_akademik=matkul['sks_akademik'],
                kode_ruangan=ruang['kode'],
                hari=hari,
                jam_mulai=jam_mulai,
                jam_selesai=jam_selesai,
                tipe_kelas=matkul['tipe_kelas']
            )
            jadwal.append(sesi)

            beban_dosen[dosen['nip']] += matkul['sks_akademik']
            jadwal_dosen[dosen['nip']].append({'hari': hari, 'jam_mulai': jam_mulai, 'jam_selesai': jam_selesai})
            jadwal_ruangan[ruang['kode']].append({'hari': hari, 'jam_mulai': jam_mulai, 'jam_selesai': jam_selesai})
            

            jumlah_mahasiswa -= ruang['kapasitas']
            index_kelas += 1

    # 🛠️ REPAIR sebelum return
    # jadwal = repair_jadwal(jadwal, dosen_list, ruang_list, hari_list, jam_list)
    return jadwal

def generate_populasi(matakuliah_list, dosen_list, ruang_list, ukuran_populasi):
    return [generate_jadwal(matakuliah_list, dosen_list, ruang_list) for _ in range(ukuran_populasi)]

In [12]:
def tampilkan_jadwal(jadwal):
    print(f"\n📅 Jadwal Mata Kuliah Terbaik ({len(jadwal)} kelas):\n")
    print(f"{'MK':<15}{'Dosen':<15}{'Ruang':<8}{'SKS':<10}{'Hari':<10}{'Jam Mulai':<12}{'Jam Selesai':<12}{'Tipe'}")
    print("-" * 60)
    # # urutkan berdasarkan kode_matkul, hari dan jam_mulai
    # for sesi in sorted(jadwal, key=lambda x: (x.kode_matkul, x.hari, x.jam_mulai)):
    #     print(f"{sesi.kode_matkul:<8}{sesi.kode_dosen:<15}{sesi.kode_ruangan:<8}{sesi.sks_akademik:<10}{sesi.hari:<10}{sesi.jam_mulai:<12}{sesi.jam_selesai:<12}{sesi.tipe_kelas}")

    # urutkan berdasarkan kode_kelas (biar keliatan jelas yang bentrok kelas mana)
    for sesi in sorted(jadwal, key=lambda x: (x.kode_ruangan, x.hari, x.jam_mulai)):
        print(f"{sesi.kode_matkul:<15}{sesi.kode_dosen:<15}{sesi.kode_ruangan:<8}{sesi.sks_akademik:<10}{sesi.hari:<10}{sesi.jam_mulai:<12}{sesi.jam_selesai:<12}{sesi.tipe_kelas}")

In [15]:
# ==================================== TESTING POPULASI + TAMPILKAN ===========================================

def cetak(jadwal):
    for sesi in jadwal:
        print(sesi)        
        
populasi = generate_populasi(matakuliah_list, dosen_list, ruang_list, 1)
for j in populasi:
    tampilkan_jadwal(j)


📅 Jadwal Mata Kuliah Terbaik (174 kelas):

MK             Dosen          Ruang   SKS       Hari      Jam Mulai   Jam Selesai Tipe
------------------------------------------------------------
TC512-AS-A     67003          FTI300  3         JUMAT     7           10          PRAKTIKUM
TC513-AS-E     67012          FTI300  3         RABU      7           10          PRAKTIKUM
TC612-H        67006          FTI300  3         RABU      10          13          PRAKTIKUM
TC715-A        67020          FTI300  3         RABU      13          16          TEORI
TC611-AS-E     68001          FTI300  3         SABTU     7           10          PRAKTIKUM
TC511-F        67002          FTI300  3         SELASA    7           10          TEORI
TC513-G        67020          FTI300  3         SELASA    10          13          PRAKTIKUM
TC615-A        67012          FTI300  3         SELASA    13          16          TEORI
TC512-AS-D     67012          FTI300  3         SENIN     7           10          PR

In [ ]:
def hitung_fitness(jadwal):
    penalti = 0
    jadwal_dosen = {}
    jadwal_ruangan = {}
    beban_dosen = {}

    # COUNTER
    hitung_dosen_bentrok = 0
    hitung_ruangan_bentrok = 0
    hitung_salah_tipe_kelas = 0
    hitung_weekend_class = 0
    hitung_pelanggaran_preferensi = 0

    for sesi in jadwal:
        # Data dosen saat ini
        info_dosen = next((d for d in dosen_list if d["nip"] == sesi.kode_dosen), None)
        info_ruangan = next((r for r in ruang_list if r["kode"] == sesi.kode_ruangan), None)
        if info_dosen:
            preferensi = info_dosen.get("preferensi", {})

            # # 1. Beban SKS Dosen
            # beban_dosen[sesi.kode_dosen] = beban_dosen.get(sesi.kode_dosen, 0) + sesi.sks_akademik
            # if beban_dosen[sesi.kode_dosen] > 12:
            #     penalti += BOBOT_PENALTI['dosen_terbebani']

            # 2. Dosen tidak boleh ganda
            if sesi.kode_dosen not in jadwal_dosen:
                jadwal_dosen[sesi.kode_dosen] = [] # bikin space kalo belum ada data berkaitan

            for sesi_lain in jadwal_dosen[sesi.kode_dosen]:
                if sesi.hari == sesi_lain['hari']:
                    if (sesi.jam_mulai in range(sesi_lain['jam_mulai']+1, sesi_lain['jam_selesai'])) or (sesi.jam_selesai  in range(sesi_lain['jam_mulai']+1, sesi_lain['jam_selesai'])) or (sesi.jam_mulai == sesi_lain['jam_mulai'] and sesi.jam_selesai == sesi_lain['jam_selesai']):
                        penalti += BOBOT_PENALTI['dosen_bentrok']
                        hitung_dosen_bentrok += 1
            
            jadwal_dosen[sesi.kode_dosen].append({'hari': sesi.hari, 'jam_mulai': sesi.jam_mulai, 'jam_selesai': sesi.jam_selesai})
        
            # 3. Ruangan tidak boleh digunakan lebih dari 1 matkul secara bersamaan
            if sesi.kode_ruangan not in jadwal_ruangan:
                jadwal_ruangan[sesi.kode_ruangan] = [] # bikin space kalo belum ada data berkaitan

            for sesi_lain in jadwal_ruangan[sesi.kode_ruangan]:
                if sesi.hari == sesi_lain['hari']:
                    if (sesi.jam_mulai in range(sesi_lain['jam_mulai']+1, sesi_lain['jam_selesai'])) or (sesi.jam_selesai  in range(sesi_lain['jam_mulai']+1, sesi_lain['jam_selesai'])) or (sesi.jam_mulai == sesi_lain['jam_mulai'] and sesi.jam_selesai == sesi_lain['jam_selesai']):
                        penalti += BOBOT_PENALTI['ruangan_bentrok']
                        hitung_ruangan_bentrok += 1

            jadwal_ruangan[sesi.kode_ruangan].append({'hari': sesi.hari, 'jam_mulai': sesi.jam_mulai, 'jam_selesai': sesi.jam_selesai})

            # 4. Salah tipe ruangan
            if sesi.tipe_kelas != info_ruangan['tipe_ruangan']:
                penalti += BOBOT_PENALTI['salah_tipe_ruangan']
                hitung_salah_tipe_kelas += 1

            # 5. Dosen tidak boleh Sabtu
            if sesi.hari == "Sabtu":
                penalti += BOBOT_PENALTI['weekend_class']
                hitung_weekend_class += 1

            # 6. Preferensi dosen
            if sesi.hari in preferensi.get("hindari_hari", []):
                penalti += BOBOT_PENALTI['melanggar_preferensi']
                hitung_pelanggaran_preferensi += 1
            for waktu in preferensi.get("hindari_jam", []):
                if waktu in range(sesi.jam_mulai, sesi.jam_selesai):
                    penalti += BOBOT_PENALTI['melanggar_preferensi']
                    hitung_pelanggaran_preferensi += 1
                    break

    # print(f"{'':<10}{'beban dosen':<40}: {beban_dosen}")
    # print(f"{'':<10}{'Bentrok Dosen':<40} : {hitung_dosen_bentrok}")
    # print(f"{'':<10}{'Bentrok Ruangan':<40} : {hitung_ruangan_bentrok}")
    # print(f"{'':<10}{'Salah Tipe Kelas':<40} : {hitung_salah_tipe_kelas}")
    # print(f"{'':<10}{'Weekend Class':<40} : {hitung_weekend_class}")
    # print(f"{'':<10}{'Melanggar Preferensi':<40} : {hitung_pelanggaran_preferensi}")
    # print(f"{'final fitness':<50} : {max(0, 100 - penalti)}")
    # return max(0, 100 - penalti)
    return 1000-penalti

In [ ]:
# ==================================== TESTING HITUNG FITNESS ===========================================

jadwal = generate_populasi(matakuliah_list, dosen_list, ruang_list, 100)
skor_fitness = [hitung_fitness(j) for j in jadwal]
print(f"min: {min(skor_fitness)}, max: {max(skor_fitness)}")

jadwal_repaired = [repair_jadwal(j, dosen_list, ruang_list) for j in jadwal]
new_skor_fitness = [hitung_fitness(j) for j in jadwal_repaired]
print(f"min: {min(new_skor_fitness)}, max: {max(new_skor_fitness)}")

In [ ]:
def roulette_selection(populasi, fitness_scores):
    total_fitness = sum(fitness_scores)
    pick = random.uniform(0, total_fitness)
    current = 0
    for individu, score in zip(populasi, fitness_scores):
        current += score
        if current > pick:
            return individu
    return random.choice(populasi)  # fallback, jika tidak ketemu

In [ ]:
def crossover(parent1, parent2):
    titik = random.randint(1, len(parent1)-1)
    child1 = parent1[:titik] + parent2[titik:]
    child2 = parent2[:titik] + parent1[titik:]
    return child1, child2

In [ ]:
def mutasi(individu, dosen_list, ruang_list, peluang_mutasi=0.1):
    pilihan_hari = ["Senin", "Selasa", "Rabu", "Kamis", "Jumat", "Sabtu"]
    pilihan_waktu = list(range(7, 19))

    for sesi in individu:
        if random.random() < peluang_mutasi:
            # Randomly mutate hari, jam, atau ruangan
            attr = random.choice(['hari', 'jam', 'ruang'])

            if attr == 'hari':
                sesi.hari = random.choice(pilihan_hari)
            elif attr == 'jam':
                jam_mulai = random.choice(pilihan_waktu)
                sesi.jam_mulai = jam_mulai
                sesi.jam_selesai = jam_mulai + sesi.sks_akademik
            elif attr == 'ruang':
                ruang = random.choice(ruang_list)
                sesi.kode_ruangan = ruang['kode']
    return individu

In [ ]:
def genetic_algorithm(matakuliah_list, dosen_list, ruang_list, ukuran_populasi=30, jumlah_generasi=100, peluang_mutasi=0.1, proporsi_elite=0.05):
    populasi = generate_populasi(matakuliah_list, dosen_list, ruang_list, ukuran_populasi)
    populasi = [repair_jadwal(j, dosen_list, ruang_list) for j in populasi]

    best_fitness_global = float('-inf')
    best_individual_global = None

    jumlah_elite = max(1, int(ukuran_populasi * proporsi_elite))

    for gen in range(jumlah_generasi):
        fitness_scores = [hitung_fitness(individu) for individu in populasi]

        # Simpan individu terbaik
        individu_elite = [individu for _, individu in sorted(zip(fitness_scores, populasi), key=lambda x: x[0], reverse=True)][:jumlah_elite]

        next_gen = individu_elite.copy()

        # Generate anak baru
        while len(next_gen) < ukuran_populasi:
            parent1 = roulette_selection(populasi, fitness_scores)
            parent2 = roulette_selection(populasi, fitness_scores)

            child1, child2 = crossover(parent1, parent2)

            child1 = mutasi(child1, dosen_list, ruang_list, peluang_mutasi)
            child2 = mutasi(child2, dosen_list, ruang_list, peluang_mutasi)

            child1 = repair_jadwal(child1, dosen_list, ruang_list)
            child2 = repair_jadwal(child2, dosen_list, ruang_list)

            next_gen.append(child1)
            if len(next_gen) < ukuran_populasi:
                next_gen.append(child2)

        populasi = next_gen

        fitness_scores = [hitung_fitness(individu) for individu in populasi]
        gen_best_fitness = max(fitness_scores)
        gen_best_individual = populasi[fitness_scores.index(gen_best_fitness)]

        if gen_best_fitness > best_fitness_global:
            best_fitness_global = gen_best_fitness
            best_individual_global = copy.deepcopy(gen_best_individual)

        print(f"[Gen {gen}] Fitness Terbaik: {gen_best_fitness}")
        print(f"            Fitness Terbaik Global: {best_fitness_global}")

    # fitness_scores = [hitung_fitness(individu) for individu in populasi]
    # print(f'list_fitness_scores {fitness_scores}')
    # best_index = fitness_scores.index(max(fitness_scores))
    # print(f'best_population: {best_index} ({max(fitness_scores)})')
    return best_individual_global

In [ ]:
# generate_populasi(matakuliah_list, dosen_list, ruang_list, 100)
# best_schedule = genetic_algorithm(matakuliah_list, dosen_list, ruang_list)
best_schedule = genetic_algorithm(matakuliah_list, dosen_list, ruang_list, ukuran_populasi=75, jumlah_generasi=500, peluang_mutasi=0.05)
# print(f'best schedule {best_schedule}')
# tampilkan_jadwal(best_schedule)